In [2]:
# webdriver-manager 설치 
# ! pip install webdriver-manager
# Webdriver import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

# Crawling import
import requests
from bs4 import BeautifulSoup
import time
import re
from datetime import datetime

### DB connect import
import pymysql
import pandas as pd
from tqdm import tqdm


### 웹드라이버 로드

In [3]:
def setup_chrome_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run Chrome in headless mode (without GUI)
    chrome_options.add_argument('--blink-settings=imagesEnabled=false') # Not load image

    # install Chrome webdriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

driver = setup_chrome_driver()

[WDM] - Downloading: 100%|██████████| 6.78M/6.78M [00:02<00:00, 3.26MB/s]


### 에펨


In [4]:
def fm_crawling(driver:webdriver, word_list : list):
    '''
    에펨코리아 사이트에서 크롤링하는 함수. 
    검색어 단어들이 리스트로 들어오면 for문으로 각 단어 결과를 파싱한 후 title, url, date, comment_cnt를 반환
    
    - input 
        word_list(list) : 연관어 리스트
        driver(webdriver) : selenium.webdriver.chrome.webdriver
    - output  
        title(str) : 제목
        url(str) : 해당 url
        date(str) : 작성일(yyyy-mm-dd 형식)
        comment_cnt(int) : 댓글수
    '''

    start = time.time()
    for word in word_list:
        url = f'https://www.fmkorea.com/search.php?act=IS&is_keyword={word}'
        driver.get(url)
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        search_in_commu = soup.find('ul', class_="searchResult")
        for search in search_in_commu: 
            if search != ' ':
                if search_in_commu.find('a') is not None:
                    title = search.find('a').get_text().split(']')
                    title = ' '.join(']'.join(title[1:]).split(' '))
                    url = f"https://www.fmkorea.com{search.find('a')['href']}"
                    date = search.find('span', class_ = 'time').get_text().split(' ')[0] # 형식 yyyy-mm-dd
                    try:
                        comment_cnt = int(search.find('em').get_text())
                    except AttributeError:   # 댓글 없는 경우
                        comment_cnt = 0
                    print(title, url, comment_cnt, date)
    end = time.time() - start
    return end

In [5]:
word_list = ['맥북', '그램', '뉴진스', '미ㅏㅇ러미낭']

fm_crawling(driver, word_list)


 서피스 프로9 vs 맥북에어13 https://www.fmkorea.com/5553381416 2 2023-03-05
 형님들 노트북 추천 부탁드리겠습니다. https://www.fmkorea.com/5553317433 2 2023-03-05
 페어링 순서에 따라 끊김이 다른게 신기하네 https://www.fmkorea.com/5553272553 0 2023-03-05
 쿠팡 애플 반품 뽑기 실패 https://www.fmkorea.com/5552910714 6 2023-03-05
 맥북프로 14인치 hdmi 질문좀 https://www.fmkorea.com/5552783246 5 2023-03-05
 서치2 대외비 후기 https://www.fmkorea.com/5552313583 0 2023-03-05
 맥북 애케플 그냥 안들었음.. https://www.fmkorea.com/5552306330 6 2023-03-05
 디x) 이 팔찌 얼마짜리 같아 https://www.fmkorea.com/5551776555 9 2023-03-05
 m2 맥북프로 14 구매완료 https://www.fmkorea.com/5551649875 1 2023-03-05
 걍 싼 노트북 아무거나 알아봐야지 https://www.fmkorea.com/5551648365 0 2023-03-05
 블랙핑크 지수 솔로곡 나오나?? https://www.fmkorea.com/5553492459 2 2023-03-06
 대학생 기숙사에서 쓸 키보드 좀 추천해주세요 https://www.fmkorea.com/5553490455 2 2023-03-06
 지금까지 컴이 왜 렉이 먹나하고 계속 찾아봣엇는데 https://www.fmkorea.com/5553453742 12 2023-03-06
 YG 새 걸그룹 BABYMONSTER 서바이벌 프로그램 - 'Last Evaluation' TEASER / 탈락자가 존재한다 https://www.fmkorea.com/5553441322 8 2023-0

11.518866777420044

### 뽐뿌


In [8]:
def pp_crawling(driver:webdriver, word_list : list):
    '''
    뽐뿌 사이트에서 크롤링하는 함수. 
    검색어 단어들이 리스트로 들어오면 for문으로 각 단어 결과를 파싱한 후 title, url, date, comment_cnt를 반환
    
    - input 
        word_list(list) : 연관어 리스트
        driver(webdriver) : selenium.webdriver.chrome.webdriver
    - output  
        title(str) : 제목
        url(str) : 해당 url
        date(str) : 작성일(yyyy-mm-dd 형식)
        comment_cnt(int) : 댓글수
    '''
    start = time.time()
    for word in word_list:
        url = f'https://www.ppomppu.co.kr/search_bbs.php?bbs_cate=2&keyword={word}'
        driver.get(url)
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        search_in_commu = soup.find_all('div', class_="content")
        for search in search_in_commu: 
            comment_cnt = int(search.find('font', class_ = 'comment-cnt').get_text())
            search.find('a').font.decompose()
            title = search.find('a').get_text()
            url = f"https://www.ppomppu.co.kr{search.find('a')['href']}"
            date = search.find('p', class_ = 'desc').select_one('span:nth-child(3)').string.split('.')
            date = '-'.join(date) # yyyy-mm-dd 형식 맞춰줌
            print(title, url, date, comment_cnt)
    times = time.time() - start
    return times

In [10]:
# test
pp_crawling(driver, word_list)

맥북 샀는데 ssd연결하려면 전용케이블 or 허브가 꼭 있어야할까요? https://www.ppomppu.co.kr/zboard/view.php?id=computer&no=671614&keyword=%B8%C6%BA%CF 2023-03-02 0
대학교 입학 시 노트북 필수인가요. https://www.ppomppu.co.kr/zboard/view.php?id=freeboard&no=8239573&keyword=%B8%C6%BA%CF 2023-03-02 18
이젠 아이패드가 싸게 느껴지네요 https://www.ppomppu.co.kr/zboard/view.php?id=tablet&no=147651&keyword=%B8%C6%BA%CF 2023-03-01 15
Tookci-67W GaN USB C 고속 충전기 ($15.86/무료) https://www.ppomppu.co.kr/zboard/view.php?id=ppomppu8&no=12726&keyword=%B8%C6%BA%CF 2023-02-28 10
애플, MFI 적용된 USB-C 케이블 폭스콘 양산 시작 https://www.ppomppu.co.kr/zboard/view.php?id=phone&no=3750271&keyword=%B8%C6%BA%CF 2023-02-28 47
다이소 맥세이프 호환용 팝핑거 https://www.ppomppu.co.kr/zboard/view.php?id=receive&no=65037&keyword=%B8%C6%BA%CF 2023-02-28 0
쿠팡 카드할인이 이상해요 ㅡ.ㅡ;; https://www.ppomppu.co.kr/zboard/view.php?id=freeboard&no=8237544&keyword=%B8%C6%BA%CF 2023-02-28 3
맥북에어 M2 vs 아이패드프로 M1+키보드 https://www.ppomppu.co.kr/zboard/view.php?id=freeboard&no=8237460&keyword=%B8%C6%BA%CF 2023-02-28 11
여

13.456708669662476

### 웃대
---

- 검색 사이트 바로 검색 안됨
- 검색어 [] 웃긴대학 내부 검색은 특수문자를 제외한 검색어가 한글 2자/영문 4자 이상이어야 검색 가능합니다.
- 해당 내용이 많으면 로딩 페이지 넘어감


In [11]:
def hu_crawling(driver:webdriver, search_word: str, related_word: str):
    '''
    humoruniv 사이트에서 크롤링하는 함수. 
    검색어 단어들이 리스트로 들어오면 for문으로 각 단어 결과를 파싱한 후 title, url, date, comment_cnt를 반환
    * search link에서 검색 안됨 -> main site에서 검색어 입력
    * 특수문자를 제외한 검색어가 한글 2자/영문 4자 이상이어야 검색 가능
    
    - input 
        word_list(list) : 연관어 리스트
        driver(webdriver) : selenium.webdriver.chrome.webdriver
    - output  
        title(str) : 제목
        url(str) : 해당 url
        date(str) : 작성일(yyyy-mm-dd 형식)
        comment_cnt(int) : 댓글수
    '''
    
    start = time.time()
    for word in word_list:
        url = "http://web.humoruniv.com/main.html" 
        driver.get(url)
        time.sleep(1)
        # 특수문자를 제외한 검색어가 한글 2자/영문 4자 이상이어야 검색 가능
        # 이부분 추가해야함

        x_path_search = '//*[@id="search_text"]'
        searchbox = driver.find_element(By.XPATH, x_path_search)
        searchbox.click()
        element = driver.find_element(By.NAME, "search_text")
        element.send_keys(f'{search_word} {related_word}')
        element.submit()

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        search_in_commu = soup.find_all('td', style ="word-break: break-all;")
        for search in search_in_commu: 
            title = search.find('a').get_text()
            url = search.find('a')['href']
            date = search.find('font').get_text().split(' ')[0]
            comment_cnt = search.find('font', color = 'green').get_text().split(' ')[1]
            print(title, url, date, comment_cnt)
    times = time.time() - start
    return times

In [13]:
# test
hu_crawling(driver, word_list)

아 맥북 사면 되는데 http://web.humoruniv.com/board/humor/read.html?table=sns&number=696502 2023-03-01 6
나 앱등이 아님 http://web.humoruniv.com/board/humor/read.html?table=pdswait&number=10189733 2023-03-01 10
아이맥...을 살까... 맥미니를... http://web.humoruniv.com/board/humor/read.html?table=pdswait&number=10185712 2023-02-27 6
맥북 쓰는데 http://web.humoruniv.com/board/humor/read.html?table=pdswait&number=10179032 2023-02-23 5
서치2보고옴 http://web.humoruniv.com/board/humor/read.html?table=workshop&number=103383 2023-02-23 4
맥북 10년만에 바꾼거 신나서 http://web.humoruniv.com/board/humor/read.html?table=pdswait&number=10177854 2023-02-23 2
오랜만에 http://web.humoruniv.com/board/humor/read.html?table=solo&number=553483 2023-02-22 6
뭔가 맥북 어플 추천 좀 해줄 수 있어? http://web.humoruniv.com/board/humor/read.html?table=pdswait&number=10172600 2023-02-20 14
맥북파셈 http://web.humoruniv.com/board/humor/read.html?table=free&number=4263007 2023-02-16 1
아이패드프로사나 맥북사나 http://web.humoruniv.com/board/humor/read.html?table=free&number=4263005 2023-02-16

UnexpectedAlertPresentException: Alert Text: 검색어 [미] 웃긴대학 내부 검색은 특수문자를 제외한 검색어가 한글 2자/영문 4자 이상이어야 검색 가능합니다.
Message: unexpected alert open: {Alert text : 검색어 [미] 웃긴대학 내부 검색은 특수문자를 제외한 검색어가 한글 2자/영문 4자 이상이어야 검색 가능합니다.}
  (Session info: headless chrome=110.0.5481.105)
Stacktrace:
Backtrace:
	(No symbol) [0x007337D3]
	(No symbol) [0x006C8B81]
	(No symbol) [0x005CB36D]
	(No symbol) [0x006318E5]
	(No symbol) [0x0061B216]
	(No symbol) [0x005F0D97]
	(No symbol) [0x005F253D]
	GetHandleVerifier [0x009AABF2+2510930]
	GetHandleVerifier [0x009D8EC1+2700065]
	GetHandleVerifier [0x009DC86C+2714828]
	GetHandleVerifier [0x007E3480+645344]
	(No symbol) [0x006D0FD2]
	(No symbol) [0x006D6C68]
	(No symbol) [0x006D6D4B]
	(No symbol) [0x006E0D6B]
	BaseThreadInitThunk [0x766E7D69+25]
	RtlInitializeExceptionChain [0x77AEBB9B+107]
	RtlClearBits [0x77AEBB1F+191]


In [35]:
# 한/영/숫자/공백 아닌 문자 입력 확인
def check_word(word):
    if re.match("^[가-힣a-zA-Z0-9 ]+$", word):
        change_word = re.sub('_', ' ', word)
        return change_word
    else:
        return False

In [39]:
# test2
word_list = ['맥북', '그램', '뉴진스', '미ㅏㅇ러미낭']
for word in word_list:
    if check_word(word) is False:
        word_list.remove(word)

hu_crawling(driver, word_list)

아 맥북 사면 되는데 http://web.humoruniv.com/board/humor/read.html?table=sns&number=696502 2023-03-01 6
나 앱등이 아님 http://web.humoruniv.com/board/humor/read.html?table=pdswait&number=10189733 2023-03-01 10
아이맥...을 살까... 맥미니를... http://web.humoruniv.com/board/humor/read.html?table=pdswait&number=10185712 2023-02-27 6
맥북 쓰는데 http://web.humoruniv.com/board/humor/read.html?table=pdswait&number=10179032 2023-02-23 5
서치2보고옴 http://web.humoruniv.com/board/humor/read.html?table=workshop&number=103383 2023-02-23 4
맥북 10년만에 바꾼거 신나서 http://web.humoruniv.com/board/humor/read.html?table=pdswait&number=10177854 2023-02-23 2
오랜만에 http://web.humoruniv.com/board/humor/read.html?table=solo&number=553483 2023-02-22 6
뭔가 맥북 어플 추천 좀 해줄 수 있어? http://web.humoruniv.com/board/humor/read.html?table=pdswait&number=10172600 2023-02-20 14
맥북파셈 http://web.humoruniv.com/board/humor/read.html?table=free&number=4263007 2023-02-16 1
아이패드프로사나 맥북사나 http://web.humoruniv.com/board/humor/read.html?table=free&number=4263005 2023-02-16

5.111664533615112

### 엠팍


In [40]:
def mp_crwaling(driver:webdriver, word_list: list):
    '''
    mlbpark 사이트에서 크롤링하는 함수. 
    검색어 단어들이 리스트로 들어오면 for문으로 각 단어 결과를 파싱한 후 title, url, date, comment_cnt를 반환
    
    - input 
        word_list(list) : 연관어 리스트
        driver(webdriver) : selenium.webdriver.chrome.webdriver
    - output  
        title(str) : 제목
        url(str) : 해당 url
        date(str) : 작성일(yyyy-mm-dd 형식)
        comment_cnt(int) : 댓글수
    '''
    
    start = time.time()
    for word in word_list:
        word = '뉴진스'
        url = f'https://mlbpark.donga.com/mp/b.php?search_select=sct&search_input={word}&x=0&y=0&select=sct&m=search&b=bullpen&query={word}'
        driver.get(url)
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        search_in_commu = soup.select('tr')
        for search in search_in_commu: 
            if search.find('a') is not None:
                title = search.find('a', class_ = 'txt').get_text()
                url = search.find('a', class_ = 'txt')['href']
                date = search.find('span', class_ = 'date').get_text()
                if ':' in date: # 당일 작성 내용은 시간으로 표시되어 오늘 날짜로 넣어줌
                    date = datetime.now().date()
                try:
                    comment_cnt = search.find('span', class_ = 'replycnt').get_text()
                    comment_cnt = int(re.sub('[\]\[]', '', comment_cnt))
                except AttributeError:
                    comment_cnt = 0
                print(title, url, date, comment_cnt)
    end = time.time() - start
    return end

In [41]:
word_list = ['맥북', '그램', '뉴진스', '니ㅏ펀']

mp_crwaling(driver, word_list)


뉴진스 노래중 가장 좋아하는 노래가 뭔가요? https://mlbpark.donga.com/mp/b.php?id=202303020078653748&p=1&b=bullpen&m=view&select=sct&query=%EB%89%B4%EC%A7%84%EC%8A%A4&user=&site=donga.com 2023-03-02 17
사실상 뉴진스 본체 https://mlbpark.donga.com/mp/b.php?id=202303020078653382&p=1&b=bullpen&m=view&select=sct&query=%EB%89%B4%EC%A7%84%EC%8A%A4&user=&site=donga.com 2023-03-02 5
뉴진스 버거광고 찍은이유 https://mlbpark.donga.com/mp/b.php?id=202303020078653286&p=1&b=bullpen&m=view&select=sct&query=%EB%89%B4%EC%A7%84%EC%8A%A4&user=&site=donga.com 2023-03-02 3
멜론차트 1위 순위가 좀 이상한데요? https://mlbpark.donga.com/mp/b.php?id=202303020078653121&p=1&b=bullpen&m=view&select=sct&query=%EB%89%B4%EC%A7%84%EC%8A%A4&user=&site=naver.com 2023-03-02 3
뉴진스가 전성기 투피엠 넘을수 있을까요? https://mlbpark.donga.com/mp/b.php?id=202303020078652098&p=1&b=bullpen&m=view&select=sct&query=%EB%89%B4%EC%A7%84%EC%8A%A4&user=&site=donga.com 2023-03-02 5
제가 요즘 배우고있는 인싸춤ㅎㄷㄷㄷㄷ.gif https://mlbpark.donga.com/mp/b.php?id=202303020078651822&p=1&b=bullpen&m=view&select=sct&query=

10.239142417907715

In [ ]:
for word in word_list:
    if check_word(word) is True:
        mp_crwaling(driver, word_list)
        fm_crawling(driver, word_list)